# Coursera Applied Data Science Capstone Project
### This repository is created as part of my Coursera capstone project

# Comparison of Food Choices of Cochin and Calicut cities

## Introduction
### Cochin and Calicut are two of the main cities in the Inidan State of Kerala. Geographically these places are located within a distance of 200Km but we can see a big difference in culture, cuisine, language etc between the people of these cities. These cities are also two of the biggest economic hubs of the state. In Calicut, service sector dominates the economy followed by industries. The city also has a strong mercantile aspect. Investments in IT is still in the beginning stage. 

### Kochi is often referred as the commercial and financial capital of Kerala. Availability of electricity, fresh water, long coastline, backwaters, good banking facilities, presence of a major port, container trans-shipment terminal, harbour terminal and an international air terminal are some of the factors which accelerated the industrial growth in the city and its adjoining district. In recent years the city has witnessed heavy investment, thus making it one of the fastest-growing second-tier metro cities in India. The district contributes 15.1% of the state GDP. Construction and manufacturing industries combined contributes 37% of the district's total GDP, and trade, tourism and hospitality industries together provides another 20%. Major business sectors include construction, manufacturing, shipbuilding, transportation/shipping, seafood and spices exports, chemical industries, information technology (IT), tourism, health services, and banking.

### This project is a comparitive study of food choices of Kochi and Calicut to understand how much the econmical and cultural differences can affect the food preferences of people. This project is useful for restaurant chains who want to start their outlets in diffrent locations to customize their offerings based on the intereset of people of the respective locations.


## Data to be used
### The analysis will be based on the location data obtained from FourSquare. First we need to understand the type of restaurants which are mostly present in these two cities.From that information, we can extract the preferred cuisines in each cities. We will also get insight into combination of cuisine and premium/cheap dining. 

### Eg. We can get the ratio of Non-veg and veg restaurants in both cities. From which we can understand whether Non-veg is preferred over veg or vice versa. From the total number of cuisines available we can see how broad is the food choices of each city. 

In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests
import numpy as np
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans
import json
from pandas.io.json import json_normalize

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import geocoder
print('Libraries Imported')

Libraries Imported


### Fetching details of Neighbourhoods

In [2]:
url = "https://www.indiatvnews.com/pincode/kerala/ernakulam/kochi"
html_data = requests.get(url).text
soup = BeautifulSoup(html_data,'html5lib')
place = []
pincode_data = soup.find('table', class_ = 'alt')
for pin_info in pincode_data.find_all('th'):
    place.append(pin_info.text)
pin_df = pd.DataFrame(columns = place)
for office_name_row in pincode_data.find_all('tr'):
    of_dat = []
    for office_name_dat in office_name_row.find_all('td'):
        of_dat.append(office_name_dat.text)
    if(len(of_dat) != 0):
        a_series = pd.Series(of_dat, index = pin_df.columns)
        pin_df = pin_df.append(a_series, ignore_index = True)


In [3]:
pin_df_kochi = pin_df[(pin_df['Taluk'] == 'Kochi')|(pin_df['Taluk'] =='Ernakulam')]
pin_df_kochi.reset_index(drop = True, inplace = True)
pin_df_kochi.head()

,Office,Taluk,District,State,Pincode
0,Palliport,Kochi,Ernakulam,KERALA,683515
1,Amballur,Ernakulam,Ernakulam,KERALA,682315
2,Andikkadavu,Kochi,Ernakulam,KERALA,682008
3,Ayyampilly,Kochi,Ernakulam,KERALA,682501
4,Azheekal,Kochi,Ernakulam,KERALA,682508


In [4]:
pin_df_kochi = pin_df_kochi[(pin_df_kochi['Office'] != 'Andikkadavu') & (pin_df_kochi['Office'] != 'Ernakulam Hindi Prachar Sabha') & (pin_df_kochi['Office'] != 'Kadamakudi') & (pin_df_kochi['Office'] != 'Kannamali') & (pin_df_kochi['Office'] != 'Kochi Naval Base') & (pin_df_kochi['Office'] != 'Kuttikkattukara') & (pin_df_kochi['Office'] !='Thiruvamkulam')]
pin_df_kochi.reset_index(drop = True, inplace = True)

In [5]:
print('The dataframe has {} Taluks and {} offices.'.format(
        len(pin_df_kochi['Taluk'].unique()),
        pin_df_kochi.shape[0]
    )
)


The dataframe has 2 Taluks and 78 offices.


In [6]:

geolocator1 = Nominatim(user_agent="tr_explorer")
lat_lon = pd.DataFrame(columns = ['latitude','longitude'])
lat_lon
for index, row in pin_df_kochi.iterrows():
    address = row['Office'] +', ' + row['District']
    location1 = geolocator1.geocode(address)
    if(location1 != None):
        print(address)
        coord = pd.Series([location1.latitude,location1.longitude],index = lat_lon.columns)
        lat_lon = lat_lon.append(coord,ignore_index = True)
    else:
        print('X', address);
#     if(location1 != None):
#         lat_lon['latitude'].iloc[index] = location1.latitude
#         lat_lon['longitude'].iloc[index] = location1.longitude
# location1 = geolocator.geocode(address)
# latitude1 = location1.latitude
# longitude1 = location1.longitude
# print('The geograpical coordinate of Kochi are {}, {}.'.format(latitude1, longitude1))
# print(location1)

Palliport, Ernakulam
Amballur, Ernakulam
Ayyampilly, Ernakulam
Azheekal, Ernakulam
Chellanam, Ernakulam
Cheranallur, Ernakulam
Chittoor ekm, Ernakulam
Cochin Special Economic Zone, Ernakulam
Edakochi, Ernakulam
Edapally North, Ernakulam
Edapally, Ernakulam
Edavanakad, Ernakulam
Elamakkara, Ernakulam
Elamkunnapuzha, Ernakulam
Ernakulam College, Ernakulam
Ernakulam, Ernakulam
Ernakulam High Court, Ernakulam
Ernakulam North, Ernakulam
Eroor Ernakulam, Ernakulam
Eroor South, Ernakulam
Eroor West, Ernakulam
Irimbanam, Ernakulam
Kadavanthara, Ernakulam
Kaipatoor, Ernakulam
Kakkanad, Ernakulam
Kakkanad West, Ernakulam
Kaloor, Ernakulam
Kochi, Ernakulam
Kochi M.G.Road, Ernakulam
Kochi Palace, Ernakulam
Kochi University, Ernakulam
Kothad, Ernakulam
Kumbalam, Ernakulam
Kumbalangi, Ernakulam
Kumbalangi South, Ernakulam
Kusumagiri, Ernakulam
Malipuram, Ernakulam
Maradu, Ernakulam
Matsyapuri, Ernakulam
Mattancherry Jetty, Ernakulam
Mattancherry, Ernakulam
Mattancherry Town, Ernakulam
Mulanthuruthy,

In [7]:
pin_df_kochi = pd.concat([pin_df_kochi,lat_lon],axis = 1)

### Creating a map of Kochi

In [8]:
# create map of Kochi using latitude and longitude values
map_kochi = folium.Map(location=[location1.latitude, location1.longitude], zoom_start=10)

# add markers to map
for lat, lng, Taluk, Office in zip(pin_df_kochi['latitude'], pin_df_kochi['longitude'], pin_df_kochi['Taluk'], pin_df_kochi['Office']):
    label = '{}, {}'.format(Office, Taluk)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kochi)  
    
map_kochi

Define Foursquare Credentials and Version

In [9]:
CLIENT_ID = 'J4I2WAJEINOGGWERVMKOIYECJDP3BL0B23524WTXGX00TYED' # your Foursquare ID
CLIENT_SECRET = 'E5YPAY4DNF2AN1HZ3KUSEJAIHJARRGPJ5T5VGFRT3CDPUNJK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
CATEGORY_ID = '4d4b7105d754a06374d81259'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: J4I2WAJEINOGGWERVMKOIYECJDP3BL0B23524WTXGX00TYED
CLIENT_SECRET:E5YPAY4DNF2AN1HZ3KUSEJAIHJARRGPJ5T5VGFRT3CDPUNJK


### Get the neighbourhood's latitude and longitude

# Explore neighbourhoods in Kochi

### Get the top 100 venues from each of the neighbourhoods in Kochi

In [33]:
def getNearbyVenues(names, latitudes, longitudes, radius=3000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            CATEGORY_ID)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues'] #[0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'], 
            v['location']['lat'], 
            v['location']['lng'],  
            v['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [34]:
kochi_venues = getNearbyVenues(names = pin_df_kochi['Office'],latitudes = pin_df_kochi['latitude'],
                                  longitudes = pin_df_kochi['longitude'])
# kochi_venues

Palliport
Amballur
Ayyampilly
Azheekal
Chellanam
Cheranallur
Chittoor ekm
Cochin Special Economic Zone
Edakochi
Edapally North
Edapally
Edavanakad
Elamakkara
Elamkunnapuzha
Ernakulam College
Ernakulam
Ernakulam High Court
Ernakulam North
Eroor Ernakulam
Eroor South
Eroor West
Irimbanam
Kadavanthara
Kaipatoor
Kakkanad
Kakkanad West
Kaloor
Kochi
Kochi M.G.Road
Kochi Palace
Kochi University
Kothad
Kumbalam
Kumbalangi
Kumbalangi South
Kusumagiri
Malipuram
Maradu
Matsyapuri
Mattancherry Jetty
Mattancherry
Mattancherry Town
Mulanthuruthy
Mulavukad
Mundamveli
Narakkal
Nayarambalam
Nazreth
Nedungad
North End
Ochanthuruth
Pachalam
Palarivattom
Palluruthy
Palluruthy South
Panampilly Nagar
Panayappilly
Perumanur
Pizhala
Poonithura
Puthuvype
Rajagiri Valley
S.Chellanam
Shanmugham Road
Thammanam
Thekkumbhagom
Thengod
Thevara
Thoppumpady
Thrikkakara
Tripunithura Fort
Tripunithura
Udayamperoor
Vaduthala
Vallarpadam
Vennala
Vyttila
Willingdon Island


In [35]:
# lat = pin_df_kochi['latitude'][0]
# lng = pin_df_kochi['longitude'][0]
# radius = 3000
# # create the API request URL
# url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
#     CLIENT_ID, 
#     CLIENT_SECRET, 
#     VERSION, 
#     lat, 
#     lng, 
#     radius, 
#     LIMIT,
#     CATEGORY_ID)
    
# # make the GET request
# results = requests.get(url).json()["response"]['venues']#[0]['items']
# results
kochi_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Palliport,10.169576,76.180716,K.R.B Homely Food,10.151695,76.175297,Indian Restaurant
1,Palliport,10.169576,76.180716,Waves Beach Restaurant,10.141860,76.178540,Kerala Restaurant
2,Palliport,10.169576,76.180716,Sea Star,10.139205,76.180516,Indian Restaurant
3,Palliport,10.169576,76.180716,Salt'n'Pepper,10.140881,76.178995,Restaurant
4,Palliport,10.169576,76.180716,Lilliput,10.141301,76.179016,Multicuisine Indian Restaurant
5,Palliport,10.169576,76.180716,P.S fast food,10.151946,76.175224,Seafood Restaurant
6,Palliport,10.169576,76.180716,Arathi's Southern Delite,10.178270,76.210625,Asian Restaurant
7,Palliport,10.169576,76.180716,Lilliput Multi Cuisine Restaurant,10.141235,76.178706,Multicuisine Indian Restaurant
8,Palliport,10.169576,76.180716,Wimbis Bake,10.195283,76.171284,Bakery
9,Palliport,10.169576,76.180716,Le Café,10.139183,76.179243,Bistro


In [36]:
print(kochi_venues.shape)
kochi_venues.head()

(2907, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Palliport,10.169576,76.180716,K.R.B Homely Food,10.151695,76.175297,Indian Restaurant
1,Palliport,10.169576,76.180716,Waves Beach Restaurant,10.141860,76.178540,Kerala Restaurant
2,Palliport,10.169576,76.180716,Sea Star,10.139205,76.180516,Indian Restaurant
3,Palliport,10.169576,76.180716,Salt'n'Pepper,10.140881,76.178995,Restaurant
4,Palliport,10.169576,76.180716,Lilliput,10.141301,76.179016,Multicuisine Indian Restaurant


In [37]:
kochi_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Amballur,6,6,6,6,6,6
Ayyampilly,16,16,16,16,16,16
Azheekal,49,49,49,49,49,49
Chellanam,8,8,8,8,8,8
Cheranallur,47,47,47,47,47,47
Chittoor ekm,1,1,1,1,1,1
Cochin Special Economic Zone,48,48,48,48,48,48
Edakochi,47,47,47,47,47,47
Edapally,49,49,49,49,49,49


### Let's find out how many unique categories can be curated from all the returned venues


In [38]:
print('There are {} uniques categories.'.format(len(kochi_venues['Venue Category'].unique())))

There are 74 uniques categories.


## Analyze each neighbourhood

In [39]:
# one hot encoding
kochi_onehot = pd.get_dummies(kochi_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
kochi_onehot['Neighborhood'] = kochi_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [kochi_onehot.columns[-1]] + list(kochi_onehot.columns[:-1])
kochi_onehot = kochi_onehot[fixed_columns]

kochi_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,BBQ Joint,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,Burger Joint,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Chettinad Restaurant,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Creperie,Cupcake Shop,Czech Restaurant,Deli / Bodega,Dessert Shop,Dhaba,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,Greek Restaurant,Halal Restaurant,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Juice Bar,Kerala Restaurant,Kosher Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mughlai Restaurant,Multicuisine Indian Restaurant,North Indian Restaurant,Pastry Shop,Pizza Place,Pub,Resort,Restaurant,Sandwich Place,Seafood Restaurant,Snack Place,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Tea Room,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant
0,Palliport,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Palliport,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Palliport,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Palliport,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
4,Palliport,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### And let's examine the new dataframe size.

In [40]:
kochi_onehot.shape

(2907, 75)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [41]:
kochi_grouped = kochi_onehot.groupby('Neighborhood').mean().reset_index()
kochi_grouped

,Neighborhood,Afghan Restaurant,American Restaurant,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Asian Restaurant,BBQ Joint,Bakery,Bistro,Brazilian Restaurant,Breakfast Spot,Burger Joint,Burrito Place,Cafeteria,Café,Cajun / Creole Restaurant,Caribbean Restaurant,Chettinad Restaurant,Chinese Restaurant,Coffee Shop,Comfort Food Restaurant,Creperie,Cupcake Shop,Czech Restaurant,Deli / Bodega,Dessert Shop,Dhaba,Diner,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Food,Food Court,Food Truck,French Restaurant,Fried Chicken Joint,Gastropub,Greek Restaurant,Halal Restaurant,Hot Dog Joint,Hotel,Ice Cream Shop,Indian Restaurant,Israeli Restaurant,Italian Restaurant,Juice Bar,Kerala Restaurant,Kosher Restaurant,Lebanese Restaurant,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Mughlai Restaurant,Multicuisine Indian Restaurant,North Indian Restaurant,Pastry Shop,Pizza Place,Pub,Resort,Restaurant,Sandwich Place,Seafood Restaurant,Snack Place,Soup Place,South Indian Restaurant,Southern / Soul Food Restaurant,Tea Room,Theme Restaurant,Tibetan Restaurant,Turkish Restaurant,Vegetarian / Vegan Restaurant
0,Amballur,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.166667,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.166667,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.166667,0.000000,0.000000,0.166667,0.000000,0.00,0.000000
1,Ayyampilly,0.000000,0.000000,0.000000,0.00,0.000000,0.125000,0.000000,0.062500,0.062500,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.062500,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.187500,0.00,0.062500,0.062500,0.062500,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.187500,0.000000,0.000000,0.000000,0.00,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,Azheekal,0.000000,0.000000,0.000000,0.00,0.020408,0.061224,0.000000,0.020408,0.020408,0.00,0.000000,0.000000,0.00,0.000000,0.204082,0.000000,0.000000,0.000000,0.000000,0.040816,0.000000,0.00,0.020408,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.020408,0.000000,0.00,0.020408,0.000000,0.020408,0.020408,0.142857,0.00,0.040816,0.020408,0.020408,0.00,0.00,0.000000,0.000000,0.020408,0.000000,0.000000,0.000000,0.000000,0.020408,0.00,0.000000,0.122449,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,Chellanam,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.250000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.250000,0.000000,0.000000,0.000000,0.125000,0.000000,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.125000,0.00,0.000000,0.000000,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.125000
4,Cheranallur,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.063830,0.000000,0.00,0.021277,0.042553,0.00,0.021277,0.085106,0.000000,0.000000,0.000000,0.000000,0.042553,0.000000,0.00,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.021277,0.00,0.000000,

In [42]:
kochi_grouped.shape

(77, 75)

#### Let's print each neighborhood along with the top 5 most common venues


In [43]:
num_top_venues = 5

for hood in kochi_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = kochi_grouped[kochi_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Amballur----
                     venue  freq
0        Indian Restaurant  0.17
1         Theme Restaurant  0.17
2                BBQ Joint  0.17
3  South Indian Restaurant  0.17
4                    Diner  0.17


----Ayyampilly----
                            venue  freq
0  Multicuisine Indian Restaurant  0.19
1               Indian Restaurant  0.19
2                      Restaurant  0.12
3                Asian Restaurant  0.12
4                          Bistro  0.06


----Azheekal----
                venue  freq
0                Café  0.20
1  Seafood Restaurant  0.14
2   Indian Restaurant  0.14
3          Restaurant  0.12
4    Asian Restaurant  0.06


----Chellanam----
                           venue  freq
0                         Bakery  0.25
1                           Café  0.25
2  Vegetarian / Vegan Restaurant  0.12
3             Chinese Restaurant  0.12
4              Indian Restaurant  0.12


----Cheranallur----
                           venue  freq
0              Indian 

#### Converting into dataframe

In [44]:
# Function to sort the venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

### Dataframe to display top 10 venues from each neighbourhood

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = kochi_grouped['Neighborhood']

for ind in np.arange(kochi_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(kochi_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amballur,Indian Restaurant,Theme Restaurant,BBQ Joint,South Indian Restaurant,Diner,Seafood Restaurant,Italian Restaurant,Mediterranean Restaurant,Lebanese Restaurant,Kosher Restaurant
1,Ayyampilly,Multicuisine Indian Restaurant,Indian Restaurant,Restaurant,Asian Restaurant,Bistro,Diner,Kerala Restaurant,Juice Bar,Italian Restaurant,Bakery
2,Azheekal,Café,Seafood Restaurant,Indian Restaurant,Restaurant,Asian Restaurant,Coffee Shop,Italian Restaurant,Kerala Restaurant,Juice Bar,Cupcake Shop
3,Chellanam,Bakery,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Indian Restaurant,Restaurant,Kosher Restaurant,Middle Eastern Restaurant,Mexican Restaurant,Mediterranean Restaurant
4,Cheranallur,Indian Restaurant,Café,Restaurant,Bakery,Vegetarian / Vegan Restaurant,Ice Cream Shop,Coffee Shop,Burger Joint,Middle Eastern Restaurant,Cafeteria


## Cluster Neighbourhoods

### Run k-means to cluster neighbourhood into 5 clusters

In [46]:
# set number of clusters
kclusters = 5

kochi_grouped_clustering = kochi_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kochi_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([4, 4, 4, 4, 0, 3, 4, 1, 4, 4])

### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [51]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
kochi_merged = pin_df_kochi
print(pin_df_kochi.shape)
# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
kochi_merged = kochi_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Office')

kochi_merged.head() # check the last columns!

(78, 7)


,Office,Taluk,District,State,Pincode,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Palliport,Kochi,Ernakulam,KERALA,683515,10.169576,76.180716,4.0,Multicuisine Indian Restaurant,Indian Restaurant,Restaurant,Asian Restaurant,Bakery,Bistro,Kerala Restaurant,Snack Place,Seafood Restaurant,Eastern European Restaurant
1,Amballur,Ernakulam,Ernakulam,KERALA,682315,9.851833,76.408795,4.0,Indian Restaurant,Theme Restaurant,BBQ Joint,South Indian Restaurant,Diner,Seafood Restaurant,Italian Restaurant,Mediterranean Restaurant,Lebanese Restaurant,Kosher Restaurant
2,Ayyampilly,Kochi,Ernakulam,KERALA,682501,10.126335,76.198371,4.0,Multicuisine Indian Restaurant,Indian Restaurant,Restaurant,Asian Restaurant,Bistro,Diner,Kerala Restaurant,Juice Bar,Italian Restaurant,Bakery
3,Azheekal,Kochi,Ernakulam,KERALA,682508,9.989438,76.241909,4.0,Café,Seafood Restaurant,Indian Restaurant,Restaurant,Asian Restaurant,Coffee Shop,Italian Restaurant,Kerala Restaurant,Juice Bar,Cupcake Shop
4,Chellanam,Kochi,Ernakulam,KERALA,682008,9.791248,76.278362,4.0,Bakery,Café,Vegetarian / Vegan Restaurant,Chinese Restaurant,Indian Restaurant,Restaurant,Kosher Restaurant,Middle Eastern Restaurant,Mexican Restaurant,Mediterranean Restaurant


### Finally, let's visualize the resulting clusters

In [52]:
print(kochi_merged.shape)
kochi_merged.dropna(inplace = True)
kochi_merged['Cluster Labels'] = kochi_merged['Cluster Labels'].astype('int32')
print(kochi_merged.shape)

(78, 18)
(77, 18)


In [55]:
# create map
map_clusters = folium.Map(location=[location1.latitude, location1.longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(kochi_merged['latitude'], kochi_merged['longitude'], kochi_merged['Office'], kochi_merged['Cluster Labels']):
#     print(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters